In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Comparison-of-basic-representations-and-operations" data-toc-modified-id="Comparison-of-basic-representations-and-operations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Comparison of basic representations and operations</a></span><ul class="toc-item"><li><span><a href="#Baseline-representation-of-partial-feature-vectors" data-toc-modified-id="Baseline-representation-of-partial-feature-vectors-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Baseline representation of partial feature vectors</a></span><ul class="toc-item"><li><span><a href="#Overhead---generation,-well-formedness,-uniquification" data-toc-modified-id="Overhead---generation,-well-formedness,-uniquification-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Overhead - generation, well-formedness, uniquification</a></span></li><li><span><a href="#Agreement" data-toc-modified-id="Agreement-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Agreement</a></span></li><li><span><a href="#Union" data-toc-modified-id="Union-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Union</a></span></li><li><span><a href="#Intersection" data-toc-modified-id="Intersection-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Intersection</a></span></li><li><span><a href="#Extension" data-toc-modified-id="Extension-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Extension</a></span></li></ul></li><li><span><a href="#Specification-array-+-value-array" data-toc-modified-id="Specification-array-+-value-array-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Specification array + value array</a></span><ul class="toc-item"><li><span><a href="#Converting-between-representations" data-toc-modified-id="Converting-between-representations-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Converting between representations</a></span></li><li><span><a href="#Operations" data-toc-modified-id="Operations-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Operations</a></span></li><li><span><a href="#Performance-evaluation" data-toc-modified-id="Performance-evaluation-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Performance evaluation</a></span><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-2.2.3.1"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li></ul></li><li><span><a href="#Matrix-extension-of-the-baseline-representation" data-toc-modified-id="Matrix-extension-of-the-baseline-representation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Matrix extension of the baseline representation</a></span><ul class="toc-item"><li><span><a href="#Agreement-testing" data-toc-modified-id="Agreement-testing-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Agreement testing</a></span></li><li><span><a href="#Union" data-toc-modified-id="Union-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Union</a></span></li><li><span><a href="#Intersection" data-toc-modified-id="Intersection-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Intersection</a></span></li><li><span><a href="#Extension" data-toc-modified-id="Extension-2.3.4"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>Extension</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-2.3.5"><span class="toc-item-num">2.3.5&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li><li><span><a href="#pytorch-and-gpus" data-toc-modified-id="pytorch-and-gpus-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span><code>pytorch</code> and gpus</a></span><ul class="toc-item"><li><span><a href="#Agreement" data-toc-modified-id="Agreement-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Agreement</a></span></li><li><span><a href="#Union" data-toc-modified-id="Union-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Union</a></span></li><li><span><a href="#Intersection" data-toc-modified-id="Intersection-2.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>Intersection</a></span></li><li><span><a href="#Extension" data-toc-modified-id="Extension-2.4.4"><span class="toc-item-num">2.4.4&nbsp;&nbsp;</span>Extension</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-2.4.5"><span class="toc-item-num">2.4.5&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></li></ul></li></ul></div>

In [2]:
import numpy as np
myint = np.int8

from vg import normalize

In [3]:
from bitarray import bitarray

In [4]:
from itertools import starmap, product

In [5]:
# from more_itertools import unique_everseen

In [6]:
from tqdm import tqdm

from joblib import Parallel, delayed, Memory

J = 30
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def par(gen_expr, j=None, backend=None, verbose=None, prefer=None):
    if j is None:
        j = J
    if backend is None:
        backend = BACKEND
    if verbose is None:
        verbose = V
    if prefer is None:
        prefer = PREFER
    return Parallel(n_jobs=j, backend=backend, verbose=verbose, prefer=prefer)(gen_expr)

def identity(x):
    return x

In [7]:
from random import choice

In [8]:
CAREFUL = False

# Overview

Goal of this notebook: find / document representations of and operations on partial feature vectors with an eye towards efficient calculation.

# Comparison of basic representations and operations

## Baseline representation of partial feature vectors

A partial feature vector $p$ on $m$ features is an element of $\{-1,0,1\}^m$, where
 - $p_i = 0$ iff feature $i$ is unspecified
 - $p_i = -1$ iff feature $i$ is specified $-$
 - $p_i = 1$ iff feature $i$ is specified $+$

Below this representation is implemented using `numpy` `int8` arrays and (usually) vectorized operations on them.

### Overhead - generation, well-formedness, uniquification

In [9]:
m = 5

In [10]:
def make_generator_vectors(num_features):
    basis_vectors = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    basis_vectors_neg = [np.zeros(num_features, dtype=myint) for each in range(num_features)]
    for i,v in enumerate(basis_vectors):
        v[i] = 1
    for i,v in enumerate(basis_vectors_neg):
        v[i] = -1
    generators = basis_vectors + basis_vectors_neg
    return generators

In [11]:
generators = make_generator_vectors(m)
generators

[array([1, 0, 0, 0, 0], dtype=int8),
 array([0, 1, 0, 0, 0], dtype=int8),
 array([0, 0, 1, 0, 0], dtype=int8),
 array([0, 0, 0, 1, 0], dtype=int8),
 array([0, 0, 0, 0, 1], dtype=int8),
 array([-1,  0,  0,  0,  0], dtype=int8),
 array([ 0, -1,  0,  0,  0], dtype=int8),
 array([ 0,  0, -1,  0,  0], dtype=int8),
 array([ 0,  0,  0, -1,  0], dtype=int8),
 array([ 0,  0,  0,  0, -1], dtype=int8)]

In [12]:
max_num_objects = 2 ** m
max_num_objects

max_num_partial_fvs = (2 + 1) ** m
max_num_partial_fvs

32

243

In [13]:
def wf_pfv(v):
    allowedValues = {-1,0,1}
    return all([x in allowedValues for x in v])

In [14]:
def make_random_pfv():
    return np.random.randint(3, size=m, dtype=myint) - 1

In [15]:
def uniquify(ndarray_iterable):
    tuples = [tuple(a) for a in ndarray_iterable]
    s = set(tuples)
    arrays = [np.array(t) for t in s]
    return arrays

In [16]:
def wf_tfv(v):
    allowedValues = {-1,1}
    return all([x in allowedValues for x in v])

In [17]:
max_num_objects
actual_num_objects = np.random.randint(max_num_objects)
# actual_num_objects = 40
actual_num_objects

assert actual_num_objects < max_num_objects

32

6

In [18]:
objects = tuple(set([tuple(np.random.randint(2, size=m)) for each in range(actual_num_objects)]))
objects = tuple(map(np.array, objects))
l = len(objects)

def zeroToMinusOne(u):
    return np.array([x if x == 1 else -1 for x in u])

objects = tuple([zeroToMinusOne(o) for o in objects])


actual_num_objects = len(objects)
actual_num_objects
objects

5

(array([-1, -1,  1,  1, -1]),
 array([-1, -1,  1, -1,  1]),
 array([-1, -1, -1, -1,  1]),
 array([-1,  1,  1,  1,  1]),
 array([-1, -1,  1,  1,  1]))

In [19]:
objectMap = np.array([objects[i] for i in range(l)])
objectMap.shape
objectMap
objectMap[0]

O = objectMap

(5, 5)

array([[-1, -1,  1,  1, -1],
       [-1, -1,  1, -1,  1],
       [-1, -1, -1, -1,  1],
       [-1,  1,  1,  1,  1],
       [-1, -1,  1,  1,  1]])

array([-1, -1,  1,  1, -1])

In [20]:
def getIndex(o, O):
    matches = [i for i,v in enumerate(O) if np.array_equal(v,o)]
    if len(matches) == 0:
        return -1
    if CAREFUL:
        assert len(matches) == 1
    return matches[0]

In [21]:
def makeExtensionVector(positive_Indices, O):
    return np.array([1 if i in positive_Indices else 0 for i in np.arange(O.shape[0])], dtype=myint)

In [22]:
makeExtensionVector([0, 4, 8], O)

array([1, 0, 0, 0, 1], dtype=int8)

### Agreement

In [23]:
def ag(x,y):
    '''
    Formula:
    (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y)), where T = 1 and F = 0
    
    Pattern:
    x = x ⟶ 1
    0 = _ ⟶ 1
    _ = 0 ⟶ 1
    _ = _ ⟶ 0
    '''
    if x == y:
        return True
    elif x == 0:
        return True
    elif y == 0:
        return True
    else:
        return False

In [24]:
def agree(u,v):
    '''
    Given two vectors u and v, returns a binary vector indicating,
    elementwise, whether u and v 'agree'.
    
    agree(u[i], v[i]) iff (u[i] == 0 or v[i] == 0) or (u[i] == v[i])
    '''
#     return np.array([True if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else False 
#                      for i in range(len(u))])
    return np.array([1 if (u[i] == 0 or v[i] == 0) or (u[i] == v[i]) else 0 
                     for i in range(len(u))], dtype=myint)

def agree_(u,v):
    '''
    Given two vectors u and v, return 1 iff u and v agree at all indices
    and 0 otherwise.
    '''
    ag = agree(u,v)
    return int(ag.all())

def agree_alt(u,v):
    '''
    Given two vectors u and v, return 1 iff u and v agree at all indices
    and 0 otherwise.
    '''
    ag = agree(u,v)
    total_agreement = np.linalg.norm(agree(u,v), 1) == m
    return int(total_agreement)
#     if total_agreement:
#         return 1.0
#     return 0.0

In [25]:
def make_agreeing_vector_pair(pred=None):
    u = make_random_pfv()
    v = make_random_pfv()
    if pred is None:
        while not agree_(u,v):
            u = make_random_pfv()
            v = make_random_pfv()
        return u,v
    while not agree_(u,v) and not pred(u,v):
        u = make_random_pfv()
        v = make_random_pfv()
    return u,v

In [26]:
num_test_pairs = int(1e5)
random_vector_pairs = [(make_random_pfv(), make_random_pfv()) for each in range(num_test_pairs)]
len(random_vector_pairs)

100000

In [27]:
for pair in random_vector_pairs:
    assert agree_(*pair) == agree_alt(*pair)

In [28]:
%%timeit

list(starmap(agree, random_vector_pairs))

1.13 s ± 2.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%%timeit

list(starmap(agree_, random_vector_pairs))

1.39 s ± 26.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit

list(starmap(agree_alt, random_vector_pairs))

2.92 s ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
num_test_pairs = int(1e5)
agreeing_vector_pairs = [make_agreeing_vector_pair() for each in range(num_test_pairs)]
len(agreeing_vector_pairs)

100000

In [39]:
pair = agreeing_vector_pairs[0]; pair

(array([0, 0, 1, 0, 0], dtype=int8), array([1, 0, 1, 0, 0], dtype=int8))

In [40]:
pair[0].dot(pair[1])

1

In [41]:
p0 = pair[0]; p0

array([0, 0, 1, 0, 0], dtype=int8)

In [42]:
np.abs(p0).sum()

1

In [36]:
# extension(p0, False)
# f = extension(p0, False)[0]; f

In [37]:
# np.dot(p0,f)

In [38]:
# del pair
# del p0
# # del f

NameError: name 'f' is not defined

### Union

The union of two partial feature vectors $u,v$ that agree should result in a partial feature vector that has every specified value in $u$, every specified value in $v$, and no other specified values.

In general, the result is at least as specified as either $u$ or $v$: when $u=v$ $u \cup v = u = v$ and $u \cup v$ is no more specified, but otherwise $u \cup v$ will be strictly more specified than either $u$ or $v$.

In [43]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs

def cup(x,y):
    '''
    Formula:
    x or y, where 1 = T, -1 = T, 0 = F
    
    Algebra:
    0 is the identity ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    (-1 and +1 are mutual inverses, but this case shouldn't occur when agree(x,y) holds)
    
    Pattern:
    x ∪ x = x
    
    0 ∪ y = y
    x ∪ 0 = x
    
    _ ∪ _ = 0  \\ <- shouldn't occur in two pfvs that agree
    '''
    if x == 0:  #if x is unspecified, return y
        return y
    elif y == 0: #if y is unspecified, return x
        return x
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

for x,y in XYs:
    ((x,y), cup(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1)

((-1, 0), -1)

((-1, 1), 0)

((0, -1), -1)

((0, 0), 0)

((0, 1), 1)

((1, -1), 0)

((1, 0), 1)

((1, 1), 1)

In [44]:
def union(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.sign(u + v)

def twoToOne(x):
    if x != 2 and x != -2:
        return x
    elif x == 2:
        return 1
    else:
        return -1

twoToOne_v = np.vectorize(twoToOne)

def union_alt(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.array(twoToOne_v(u + v), dtype=myint)

def union_alt2(u, v):
    if CAREFUL:
        assert agree_(u,v)
    s = u + v
    return np.trunc( np.sqrt(np.abs(s)) ) * np.sign(s,dtype=myint)

def union_alt3(u, v):
    if CAREFUL:
        assert agree_(u,v)
    w = u.copy()
    for i,x in enumerate(v):
        if x != 0:
            w[i] = x
    return w

def union_alt4(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.array([cup(u[i],v[i]) for i in range(m)], dtype=myint)

cup_v = np.vectorize(cup)

def union_alt5(u, v):
    if CAREFUL:
        assert agree_(u,v)
    return np.array(cup_v(u,v), dtype=myint)

In [45]:
test_pair = choice(agreeing_vector_pairs)
test_pair

union(*test_pair)
union_alt(*test_pair)
union_alt2(*test_pair)
union_alt3(*test_pair)
union_alt4(*test_pair)
union_alt5(*test_pair)

(array([-1,  0,  0,  0,  0], dtype=int8),
 array([ 0,  1,  1, -1,  0], dtype=int8))

array([-1,  1,  1, -1,  0], dtype=int8)

array([-1,  1,  1, -1,  0], dtype=int8)

array([-1.,  1.,  1., -1.,  0.], dtype=float16)

array([-1,  1,  1, -1,  0], dtype=int8)

array([-1,  1,  1, -1,  0], dtype=int8)

array([-1,  1,  1, -1,  0], dtype=int8)

In [46]:
%%timeit

list(starmap(union, agreeing_vector_pairs));

81.5 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit

list(starmap(union_alt, agreeing_vector_pairs));

1.54 s ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit

list(starmap(union_alt2, agreeing_vector_pairs));

345 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%%timeit

list(starmap(union_alt3, agreeing_vector_pairs));

671 ms ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%%timeit

list(starmap(union_alt4, agreeing_vector_pairs));

1.1 s ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%%timeit

list(starmap(union_alt5, agreeing_vector_pairs));

1.65 s ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Intersection

The intersection of two partial feature vectors $u,v$ should result in a partial feature vector that has every specified value that is specified in both $u$ and $v$ and where $u$ and $v$ agree, and no other specified values.

In general, the result is no more specified than either $u$ or $v$: when $u=v$ $u \cap v = u = v$ and $u \cap v$ is no less specified, but otherwise $u \cap v$ will be strictly less specified than either $u$ or $v$.

In [52]:
XYs = tuple(product((-1,0,1), (-1,0,1)))
XYs 
    
def cap(x,y):
    '''
    Algebra:
    0 is the annihilating element ∀x ∈ {-1,0,+1}
    x is its own identity ∀x ∈ {-1,0,+1}
    -1 and +1 annihilate each other
    
    Pattern:
    x ∩ x = x
    
    0 ∩ _ = 0
    _ ∩ 0 = 0
    
    _ ∩ _ = 0
    '''
    if x == 0: #if x is unspecified, return 0
        return 0
    elif y == 0: #if y is unspecified, return 0
        return 0
    elif x == y: #if both are specified and the same, return their common value
        return x
    else: #otherwise return 0
        return 0

def foo(x,y):
    return np.sign( (x == y) * (x + y) )

def bar(x,y):
    return (x == y) * (x + y) * 0.5

def baz(x,y):
    return (x == y) * int((x + y) / 2)

for x,y in XYs:
#     ((x,y), cap(x,y))
    ((x,y), cap(x,y), foo(x,y), bar(x,y), baz(x,y))

((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1))

((-1, -1), -1, -1, -1.0, -1)

((-1, 0), 0, 0, 0.0, 0)

((-1, 1), 0, 0, 0.0, 0)

((0, -1), 0, 0, 0.0, 0)

((0, 0), 0, 0, 0.0, 0)

((0, 1), 0, 0, 0.0, 0)

((1, -1), 0, 0, 0.0, 0)

((1, 0), 0, 0, 0.0, 0)

((1, 1), 1, 1, 1.0, 1)

In [53]:
def intersection(u, v):
    return np.sign(  np.equal(u, v) * (u + v) )

def intersection_alt(u, v):
    return np.array([cap(u[i],v[i]) for i in range(m)], dtype=myint)

def intersection_alt2(u, v):
    return np.array(np.equal(u, v) * (u + v) * 0.5, dtype=myint)

def intersection_alt3(u, v):
    return np.array([bar(u[i], v[i]) for i in range(m)], dtype=myint)

In [54]:
test_pair = choice(random_vector_pairs)
test_pair

intersection(*test_pair)
intersection_alt(*test_pair)
intersection_alt2(*test_pair)
intersection_alt3(*test_pair)

(array([ 1,  0,  1,  1, -1], dtype=int8),
 array([ 1,  0, -1,  0,  1], dtype=int8))

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 0, 0, 0, 0], dtype=int8)

In [55]:
for pair in random_vector_pairs:
    assert np.array_equal(intersection(*pair), intersection_alt(*pair)), 'Agreement failure on {0}'.format(pair)
    assert np.array_equal(intersection_alt2(*pair), intersection_alt(*pair)), 'Agreement failure on {0}'.format(pair)
    assert np.array_equal(intersection_alt3(*pair), intersection_alt(*pair)), 'Agreement failure on {0}'.format(pair)

In [56]:
%%timeit

list(starmap(intersection, random_vector_pairs));

177 ms ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%%timeit

list(starmap(intersection_alt, random_vector_pairs));

1.16 s ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%%timeit

list(starmap(intersection_alt2, random_vector_pairs));

298 ms ± 478 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%%timeit

list(starmap(intersection_alt3, random_vector_pairs));

1.87 s ± 3.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Extension

In [60]:
def extension(v, O, asIndexVector=True):
    '''
    The extension of a partial feature vector v is the set of object vectors
    (= fully specified, or 'total' feature vectors) that 'agree' with it.
    '''
    matches = tuple([o for o in O if agree_(v,o)])
#     matches = tuple([o for o in objects if agree(v,o).all()])
#     matches = np.array([1.0 if np.linalg.norm(agree(v,o), 1) == num_features else 0.0 for o in objects])
    if asIndexVector:
        return makeExtensionVector([getIndex(o, O) for o in matches], O)
    return matches

In [61]:
def in_extension(s, o):
    '''
    Given a partial feature vector s and a fully specified object vector o,
    returns True iff o ∈ ⟦s⟧ and False otherwise.
    '''
    if np.array_equal( s, np.zeros(s.shape) ):
        return True
    
    s_ = np.abs(s)
    o_ = s_ * o
    s_normed = normalize(s)
    o_normed = normalize(o_)
    pr = np.dot(s_normed, o_normed)
    return np.isclose(1.0, pr)

def extension_alt(s, O):
    extensionVector = np.array([1 if in_extension(s, o) else 0 for o in O], dtype=myint)
    return extensionVector

In [62]:
def in_extension_alt2(s, o):
    '''
    Given a partial feature vector s and a fully specified object vector o,
    returns True iff o ∈ ⟦s⟧ and False otherwise.
    '''
    if np.array_equal( s, np.zeros(s.shape) ):
        return True
    
    s_ = np.abs(s)
    mag = np.sum(s_)
    s_prime = s_ / mag
#     o_ = s_ * o
#     s_normed = normalize(s)
#     o_normed = normalize(o_)
#     pr = np.dot(s_normed, o_normed)
#     pr = np.dot(s_normed, o)
#     pr = np.dot(s_, o)
    pr = np.dot(s_prime, o)
#     return np.isclose(1.0, pr)
#     return pr == mag
    return pr#, mag

def extension_alt2(s, O):
    extensionVector = np.array([1 if in_extension_alt(s, o) else 0 for o in O], dtype=myint)
    return extensionVector

In [63]:
def ramp(M):
    return np.heaviside(M-1, 1).astype(myint)

def primed(p):
    mag_p = np.sum(np.abs(p))
    return p / mag_p

def unramped_in_extension_alt3(s,o):
    p = s
    return np.dot(o, primed(p))

def in_extension_alt3(s, o):
    p = s
    return ramp( np.dot(o, primed(p)) )

def extension_alt3(s, O):
    if np.array_equal(s, np.zeros(s.shape)):
        return np.ones((l,), dtype=myint)
    p = s
#     mag_p = np.sum(np.abs(p))
#     p_prime = p / mag_p
    return ramp( np.dot(O, primed(p)) )

def unramped_extension_alt3(s, O):
    if np.array_equal(s, np.zeros(s.shape)):
        return np.ones((l,), dtype=myint)
    p = s
#     mag_p = np.sum(np.abs(p))
#     p_prime = p / mag_p
    return np.dot(O, primed(p))

In [64]:
p = make_random_pfv()
while len(extension(p, O, False)) == 0:
    p = make_random_pfv()
print( 'p = {0}'.format(p) )
print( '⟦p⟧ = \n{0}'.format(extension(p, O, False)) )
p_x0 = extension(p, O, False)[0];
print( '⟦p⟧_0 = \n{0}'.format(p_x0) )
print( '⟦p⟧_0 ∈ ⟦p⟧? {0}'.format(in_extension(p, p_x0)) )
# in_extension_alt(p, p_x0)
np.array( [ramp( np.dot(p_xi, primed(p)) ) for p_xi in extension(p, O, False)] )
print(' ')

mag_p = np.sum(np.abs(p))
print( '|abs(p)| = {0}'.format( mag_p ) )
p_prime = p / mag_p
print( "p' = {0}".format( p_prime  ) )
ext_p = np.array(extension(p, O, True))
print( '⟦p⟧ = \n{0}'.format(ext_p) )
A = extension(p, O, False)
print( 'A = ⟦p⟧ = \n{0}'.format(np.array(A)) )
A_dot_p = np.dot(A, p)
print('A⸱p = {0}'.format( A_dot_p ))
print( 'V = \n{0}'.format(O) )
# print( 'unramped_3 ⟦p⟧:\n{0}'.format(unramped_extension_alt3(p)) )
V_dot_p = np.dot(objectMap, p)
print("V⸱p = \n{0}".format( V_dot_p))
V_dot_p_prime = np.dot(objectMap, p_prime)
print( "V⸱p' = \n{0}".format(V_dot_p_prime) )

def ramp(M):
    return np.heaviside(M-1, 1).astype(myint)
ramp(V_dot_p_prime)
np.array_equal(ext_p, ramp(V_dot_p_prime))
# np.heaviside( np.dot(objectMap, p/(np.sum(np.abs(p)))) )

p = [ 0  0  0  0 -1]
⟦p⟧ = 
(array([-1, -1,  1,  1, -1]),)
⟦p⟧_0 = 
[-1 -1  1  1 -1]
⟦p⟧_0 ∈ ⟦p⟧? True


array([1], dtype=int8)

 
|abs(p)| = 1
p' = [ 0.  0.  0.  0. -1.]
⟦p⟧ = 
[1 0 0 0 0]
A = ⟦p⟧ = 
[[-1 -1  1  1 -1]]
A⸱p = [1]
V = 
[[-1 -1  1  1 -1]
 [-1 -1  1 -1  1]
 [-1 -1 -1 -1  1]
 [-1  1  1  1  1]
 [-1 -1  1  1  1]]
V⸱p = 
[ 1 -1 -1 -1 -1]
V⸱p' = 
[ 1. -1. -1. -1. -1.]


array([1, 0, 0, 0, 0], dtype=int8)

True

In [65]:
def specifiable_zero_indices(p, ext_p):
    '''
    Given p and A::(n,m) = ⟦p⟧:
    
    If p_j = 0 and ∀i A_{i,j} = k≠0, then
    p_j is unspecified (i.e. p_j = 0) but 
    can be set to k and yield a co-extensive 
    and more specific pfv p'. (NB: p' entails 
    p.)
    
    This function returns a list of 0-valued 
    indices of p that can be specified, but not 
    what the common value at that index is.
    
    Correctly specifying any one or any combination
    of the indices in this list of indices will result
    in a more specific vector than p that is coextensive.
    
    From this list, you can construct (or count) all of the
    more specified pfvs that are coextensive with p.
    '''
    A = ext_p
    n = A.shape[0]
    n_opp = -1.0 * n
#     zeros = np.nonzero(p)[0]
    zero_indices = np.array(tuple(  set(range(len(p))) - set(np.nonzero(p)[0])  ), dtype=myint)
    specifiable_indices = set()
    for j in zero_indices:
        j_col_sum = np.sum(A[:,j])
        if j_col_sum == n or j_col_sum == n_opp:
            specifiable_indices.add(j)
    return specifiable_indices

In [66]:
p

array([ 0,  0,  0,  0, -1], dtype=int8)

In [67]:
np.array(extension(p, O, False))
np.array(extension(p, O, False)).shape

array([[-1, -1,  1,  1, -1]])

(1, 5)

In [68]:
p
np.nonzero(p)[0]
set(np.nonzero(p)[0])
np.array( tuple(
    set(range(len(p))) - set(np.nonzero(p)[0])
), dtype=myint )

array([ 0,  0,  0,  0, -1], dtype=int8)

array([4])

{4}

array([0, 1, 2, 3], dtype=int8)

In [69]:
?extension_alt3

In [70]:
# p = make_random_pfv()
extension(p, O)
extension_alt(p, O)
extension_alt3(p, O)
np.array_equal(extension_alt(p, O), extension_alt3(p, O))

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 0, 0, 0, 0], dtype=int8)

array([1, 0, 0, 0, 0], dtype=int8)

True

In [71]:
extension_alt3(p, O)

array([1, 0, 0, 0, 0], dtype=int8)

In [72]:
O

array([[-1, -1,  1,  1, -1],
       [-1, -1,  1, -1,  1],
       [-1, -1, -1, -1,  1],
       [-1,  1,  1,  1,  1],
       [-1, -1,  1,  1,  1]])

In [73]:
num_test_pairs = int(1e5)
random_vectors = [make_random_pfv() for each in range(num_test_pairs)]
len(random_vectors)

100000

In [74]:
def extension_test(v, O):
    foo = extension(v, O)
    bar = extension_alt(v, O)
    baz = extension_alt3(v, O)
    if not np.array_equal(foo, bar):
        raise Exception('1. Disagreement on {0}: {1} vs. {2}'.format(v, extension(v, O), extension_alt(v, O)))
    if not np.array_equal(foo, baz):
        raise Exception('2. Disagreement on {0}: {1} vs. {2}'.format(v, extension(v, O), extension_alt3(v, O)))
    return True

In [75]:
par(delayed(extension_test)(v, O) for v in random_vectors)

# for p in tqdm(random_vectors):
#     assert np.array_equal(extension(p), extension_alt(p)), 'Disagreement on {0}: {1} vs. {2}'.format(p, extension(p), extension_alt(p))

[Parallel(n_jobs=30)]: Using backend MultiprocessingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=30)]: Batch computation too fast (0.0052s.) Setting batch_size=76.
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=30)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=30)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=30)]: Done  53 tasks      | elapsed:    0.1s
[Parallel(n_jobs=30)]: Batch computation too fast (0.0620s.) Setting batch_size=490.
[Parallel(n_jobs=30)]: Done 668 tasks      | elapsed:    0.2s
[Parallel(n_jobs=30)]: Done 1960 tasks      | elapsed:    0.4s
[Parallel(n_jobs=30)]: Done 3252 tasks      | elapsed:    0.5s
[Parallel(n_jobs=30)]: Done 5110 tasks      | elapsed:    0.7s
[Parallel(n_jobs=30)]: Done 14420 tasks      | elapsed:    2.2s
[Parallel(n_jobs=30)]: Batch computation too slow (2.0095s.) Setting batch_size=245.
[Parallel(n_jobs=30)]: Done 24710

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [76]:
%%timeit

list(map(lambda v: extension(v, O), random_vectors))

9.84 s ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%%timeit

list(map(lambda v: extension_alt(v, O), random_vectors))

21.2 s ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%%timeit

list(map(lambda v: extension_alt3(v, O), random_vectors))

1.41 s ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
# agree_mat(p, objectMap)

## Specification array + value array

This representation of a partial feature vector $p$ uses two bit sequences, $s$ and $v$
 - $s_i = 0$ iff $p_i = 0$ and is otherwise $1$
 - $v_i = 0$ if $p_i = -1$
 - $v_i = 1$ if $p_i = 1$

Note that the value of $v_i$ is unspecified if $p_i = 0$.

### Converting between representations

In [80]:
spec_cb = {-1:bitarray('1'),
            0:bitarray('0'),
            1:bitarray('1')}
val_cb = {-1:bitarray('0'),
           0:bitarray('0'),
           1:bitarray('1')}

In [81]:
def pfv_to_sv(pfv):
    s = bitarray()
    s.encode(spec_cb, list(pfv))
    v = bitarray()
    v.encode(val_cb, list(pfv))
    return s,v

def sv_to_pfv(s,v):
#     print('s,v = {0}, {1}'.format(s.to01(), v.to01()))
    specified = np.array(s.tolist(), dtype=myint)
    values = np.array(v.tolist(), dtype=myint)
#     print('\ts,v = {0}, {1}'.format(specified, values))
    for i in range(m):
        if specified[i] != 0:
            specified[i] = 1 if values[i] else -1
#     print('\ts = {0}'.format(specified))
    return specified

In [82]:
num_test_pairs = int(1e5)
random_vectors = [make_random_pfv() for each in tqdm(range(num_test_pairs))]
len(random_vectors)

100%|██████████| 100000/100000 [00:00<00:00, 298269.47it/s]


100000

In [83]:
for p in tqdm(random_vectors):
    s,v = pfv_to_sv(p)
    p_prime = sv_to_pfv(s,v)
    assert np.array_equal(p, p_prime), 'Conversion failure on {0}'.format(pair)

100%|██████████| 100000/100000 [00:02<00:00, 35563.55it/s]


### Operations

If $p,q$ are two partial feature vectors and $(s^p, v^p), (s^q, v^q)$ are their associated specification and value bitvectors, then:

We can define an element-wise agree operation by pattern matching:
```
agree((s^p_i,v^p_i), (s^q_i,v^q_i)):
    (s_w, s_x) = (s_w, s_x) ⟶ 1
    (0,0)      = (_, _)     ⟶ 1
    (_, _)     = (0,0)      ⟶ 1
    _          = _          ⟶ 0
```
Or perhaps more clearly by Boolean formula:
 - $\text{agree}(s^p_i,v^p_i,s^q_i,v^q_i) = (\neg s^p_i \lor \neg s^q_i) \lor ((s^p_i \land s^q_i) \land (v^p_i \iff v^q_i)) $

We can define an element-wise union operation (assuming agreement holds) by pattern matching:
```
Assuming agree(p,q) holds:

cup((s^p_i,v^p_i), (s^q_i,v^q_i)):
    (s_w, s_x) ∪ (s_w, s_x) = (s_w, s_x)
    (0,0)      ∪ (s_y, s_z) = (s_y, s_z)
    (s_w, s_x) ∪ (0,0)      = (s_w, s_x)
    _          ∪ _          = (0,0)
```
Or again, more clearly by Boolean formula:
 - $\text{cup}(s^p_i,v^p_i,s^q_i,v^q_i) = (s^p_i \lor s^q_i, v^p_i \lor v^q_i)$

I.e. we can take the `bitwise or` of respective specification vectors and value vectors to get the specification and value vector of the union of two partial feature vectors.

We can define an element-wise intersection operation by pattern matching:
```
cap((s^p_i,v^p_i), (s^q_i,v^q_i)):
    (s_w, s_x) ∩ (s_w, s_x) = (s_w, s_x)
    (0,0)      ∩ (_, _)     = (0,0)
    (_, _)     ∩ (0,0)      = (0,0)
    _          ∩ _          = (0,0)
```
...or by Boolean formula
 - $\text{cap}(s^p_i,v^p_i,s^q_i,v^q_i) = ((s^p_i \land s^q_i) \land (v^p_i \iff v^q_i), v^p_i \land v^q_i)$

In [84]:
def xor(p,q):
    return (p & ~q) | (~p & q)

def ifthen(p,q):
    return ~p | q

def iff(p,q):
    return ifthen(p,q) & ifthen(q,p)

assert xor(bitarray('0011'), bitarray('0101')) == bitarray('0110')
assert ifthen(bitarray('0011'), bitarray('0101')) == bitarray('1101')
assert iff(bitarray('0011'), bitarray('0101')) == bitarray('1001')

In [85]:
def agree_ba(s_p, v_p, s_q, v_q):
    return (~s_p | ~s_q) | ((s_p & s_q) & iff(v_p, v_q))

def agree_ba_(s_p, v_p, s_q, v_q):
    return agree_ba(s_p, v_p, s_q, v_q).all()

def union_ba(s_p, v_p, s_q, v_q):
    return s_p | s_q, v_p | v_q

def intersection_ba(s_p, v_p, s_q, v_q):
    return (s_p & s_q) & iff(v_p, v_q), v_p & v_q

Below we test that they have the same behavior as the baseline representation and operations:

In [86]:
for p,q in tqdm(agreeing_vector_pairs):
    s_p, v_p = pfv_to_sv(p)
    s_q, v_q = pfv_to_sv(q)
    assert np.array_equal(agree(p,q), np.array(list(agree_ba(s_p, v_p, s_q, v_q)), dtype=myint))
    assert np.array_equal(union(p,q), sv_to_pfv(*union_ba(s_p, v_p, s_q, v_q)))

100%|██████████| 100000/100000 [00:06<00:00, 15366.85it/s]


In [87]:
for p,q in tqdm(random_vector_pairs):
    s_p, v_p = pfv_to_sv(p)
    s_q, v_q = pfv_to_sv(q)
    assert np.array_equal(agree(p,q), np.array(list(agree_ba(s_p, v_p, s_q, v_q)), dtype=myint))
    assert np.array_equal(intersection(p,q), sv_to_pfv(*intersection_ba(s_p, v_p, s_q, v_q)))

100%|██████████| 100000/100000 [00:07<00:00, 14191.58it/s]


In [88]:
def in_extension_ba(s_p, v_p, s_o, v_o):
    return agree_ba_(s_p, v_p, s_o, v_o)

### Performance evaluation

Now we compare timing:

In [89]:
agreeing_vector_pairs_ba = [(pfv_to_sv(u)[0], pfv_to_sv(u)[1], pfv_to_sv(v)[0], pfv_to_sv(v)[1]) for u,v in agreeing_vector_pairs]

In [90]:
random_vector_pairs_ba = [(pfv_to_sv(u)[0], pfv_to_sv(u)[1], pfv_to_sv(v)[0], pfv_to_sv(v)[1]) for u,v in random_vector_pairs]

In [91]:
def unpack_pfv_pair(pair):
    p = pair[0]
    q = pair[1]
    s_p, v_p = pfv_to_sv(p)
    s_q, v_q = pfv_to_sv(q)
    return (s_p, v_p, s_q, v_q)

# def unpack_sv_pair_pair(sv_pair_pair):
#     s_p, v_p = sv_pair_pair[0][0], sv_pair_pair[0][1]
#     s_q, v_q = sv_pair_pair[1][2], sv_pair_pair[1][1]
#     return (s_p, v_p, s_q, v_q)

In [92]:
%%timeit

list(starmap(agree, random_vector_pairs))

1.13 s ± 2.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
%%timeit

list(starmap(agree_ba, random_vector_pairs_ba))

173 ms ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [94]:
%%timeit

list(starmap(union, agreeing_vector_pairs))

81.3 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [95]:
%%timeit

list(starmap(union_ba, agreeing_vector_pairs_ba))

49 ms ± 18 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:
%%timeit

list(starmap(intersection, random_vector_pairs))

177 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [97]:
%%timeit

list(starmap(intersection_ba, random_vector_pairs_ba))

150 ms ± 45.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Conclusion

In the baseline representation:
 - `agreement` checking is very expensive, taking ≈10x longer than `union`ing and ≈5x longer than `intersect`ing

In the bitarray representation:
 - `agreement` checking and `intersection` take comparably long, and both take about 5x longer than `union`

`agreement` checking is about 5x faster with bitarrays than with the baseline representation, `intersection` is about 1.25x faster, and `union` is about 2x faster.

## Matrix extension of the baseline representation

In [98]:
random_stack_list = random_vector_pairs[:3]; random_stack_list

[(array([ 1,  0,  0, -1,  1], dtype=int8),
  array([ 1, -1,  0, -1,  1], dtype=int8)),
 (array([0, 0, 0, 0, 0], dtype=int8), array([-1,  0,  1,  1,  0], dtype=int8)),
 (array([ 0, -1,  0, -1, -1], dtype=int8),
  array([-1,  1,  0, -1, -1], dtype=int8))]

In [99]:
random_vector_pairs[:10]

[(array([ 1,  0,  0, -1,  1], dtype=int8),
  array([ 1, -1,  0, -1,  1], dtype=int8)),
 (array([0, 0, 0, 0, 0], dtype=int8), array([-1,  0,  1,  1,  0], dtype=int8)),
 (array([ 0, -1,  0, -1, -1], dtype=int8),
  array([-1,  1,  0, -1, -1], dtype=int8)),
 (array([ 1,  1,  0, -1,  1], dtype=int8),
  array([-1, -1,  0,  0,  1], dtype=int8)),
 (array([-1,  0,  0, -1, -1], dtype=int8),
  array([ 0,  0, -1,  0,  0], dtype=int8)),
 (array([ 0,  0, -1,  0, -1], dtype=int8),
  array([-1,  1,  0,  1, -1], dtype=int8)),
 (array([ 1,  1,  0, -1, -1], dtype=int8),
  array([ 0,  0,  1, -1,  1], dtype=int8)),
 (array([-1, -1,  1,  0, -1], dtype=int8),
  array([ 1,  1, -1,  0,  1], dtype=int8)),
 (array([ 1,  0, -1, -1,  1], dtype=int8),
  array([-1,  0,  0, -1, -1], dtype=int8)),
 (array([1, 1, 0, 1, 0], dtype=int8), array([0, 0, 0, 1, 0], dtype=int8))]

In [100]:
first = lambda seq: seq[0]
second = lambda seq: seq[1]

stack_a, stack_b = list(map(first, random_vector_pairs)), list(map(second, random_vector_pairs))
random_pair_stack_a, random_pair_stack_b = np.array(stack_a), np.array(stack_b)
random_pair_stack_a.dtype
random_pair_stack_b.dtype

dtype('int8')

dtype('int8')

In [101]:
stack_a, stack_b = list(map(first, agreeing_vector_pairs)), list(map(second, agreeing_vector_pairs))
agreeing_pair_stack_a, agreeing_pair_stack_b = np.array(stack_a), np.array(stack_b)
agreeing_pair_stack_a.dtype
agreeing_pair_stack_b.dtype

dtype('int8')

dtype('int8')

### Agreement testing

In [102]:
random_pair_stack_a.shape
n = random_pair_stack_a.shape[0]

(100000, 5)

In [103]:
list(starmap(agree_, random_vector_pairs));

In [104]:
vector_agree__results = np.array([agree_(random_pair_stack_a[i],random_pair_stack_b[i]) for i in range(n)])
vector_agree__results.shape

(100000,)

In [105]:
def agree_mat(A,B):
    '''
    Given two matrices A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = np.equal(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
    result = np.prod(ag, axis=-1, dtype=myint)
    return result

In [106]:
matrix_agree_result = agree_mat(random_pair_stack_a, random_pair_stack_b)
matrix_agree_result.shape

(100000,)

In [107]:
np.array_equal(matrix_agree_result, vector_agree__results)

True

In [108]:
for i in range(n):
    u = random_pair_stack_a[i]
    v = random_pair_stack_b[i]
    assert agree_(u,v) == agree_mat(u,v), '{0}, {1} -> {2} vs. {3}'.format(u,v, agree_(u,v), agree_mat(u,v, True))

### Union

In [109]:
vector_union_results = np.array([union(agreeing_pair_stack_a[i],agreeing_pair_stack_b[i]) for i in range(n)])
vector_union_results.shape

(100000, 5)

In [110]:
union(agreeing_pair_stack_a, agreeing_pair_stack_b)

array([[ 1,  0,  1,  0,  0],
       [ 1,  0,  1,  1, -1],
       [ 1, -1,  1, -1, -1],
       ...,
       [-1, -1, -1,  0,  1],
       [-1,  1,  0,  1,  1],
       [-1,  1,  1, -1,  0]], dtype=int8)

In [111]:
np.array_equal(vector_union_results, union(agreeing_pair_stack_a, agreeing_pair_stack_b) )

True

### Intersection

In [112]:
vector_intersection_results = np.array([intersection(random_pair_stack_a[i],random_pair_stack_b[i]) for i in range(n)])
vector_intersection_results.shape

(100000, 5)

In [113]:
intersection(agreeing_pair_stack_a, agreeing_pair_stack_b)

array([[ 0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  1,  0, -1],
       ...,
       [ 0,  0, -1,  0,  0],
       [-1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0]], dtype=int8)

In [114]:
np.array_equal(vector_intersection_results, intersection(random_pair_stack_a, random_pair_stack_b) )

True

### Extension

Using `agree_mat` we can define a faster method for calculating extensions:

In [115]:
def extension_(pfv, O):
    return agree_mat(pfv, O)

In [116]:
p = make_random_pfv()
p
' '
extension(p, O)
extension_alt(p, O)
extension_(p, O)

array([1, 0, 0, 1, 1], dtype=int8)

' '

array([0, 0, 0, 0, 0], dtype=int8)

array([0, 0, 0, 0, 0], dtype=int8)

array([0, 0, 0, 0, 0], dtype=int8)

In [117]:
for p in tqdm(random_vectors):
    assert np.array_equal(extension(p, O), extension_(p, O)), 'Disagreement on {0}: {1} vs. {2}'.format(p, extension(p, O), extension_alt(p, O))

100%|██████████| 100000/100000 [00:12<00:00, 8017.08it/s]


In [118]:
%%timeit

list(map(lambda v: extension(v, O), random_vectors))

9.85 s ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [119]:
%%timeit

list(map(lambda v: extension_(v,O), random_vectors))

1.19 s ± 4.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Conclusion

The `union` and `intersection` operations extend naturally to vector stacks (matrices), and vectorized versions of the boolean formulas discovered during exploration of bitarrays make a vectorized version of `agreement` easily definable.

All three operations are enormously faster than either the baseline representation or the bitarray methods.

The vectorized version of agreement naturally permits a moderately (≈5-6x) faster method of calculating the extension of a partial feature vector.

## `pytorch` and gpus

The next logical question is whether gpus can usefully accelerate computation...

In [120]:
import torch

In [128]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
gpu_int8_ttype = torch.cuda.CharTensor
gpu_int16_ttype = torch.cuda.ShortTensor
my_ttype = gpu_int8_ttype
my_dtype = torch.uint8
def t(ndarray):
    if ndarray.dtype == myint:
        return torch.tensor(ndarray.astype(np.int16)).type(my_ttype)
    return torch.tensor(ndarray).type(my_ttype)

In [129]:
objects_t = t(np.array(objects))
objectMap_t = t(objectMap)
O_t = objectMap_t

### Agreement

In [123]:
p

array([-1, -1,  0,  1,  1], dtype=int8)

In [124]:
q

array([1, 0, 1, 1, 0], dtype=int8)

In [125]:
np.equal(p,q)
torch.eq(t(p), t(q))

array([False, False, False,  True, False])

tensor([0, 0, 0, 1, 0], dtype=torch.uint8)

In [126]:
tens = torch.tensor([1,1,1])
tens = tens.type(torch.cuda.ByteTensor)
tens
tens.type()
tens.all()

tensor([1, 1, 1], dtype=torch.uint8)

'torch.cuda.ByteTensor'

tensor(1, dtype=torch.uint8)

In [130]:
quux = torch.tensor([[1,1,1],[1,0,1]])
quux = quux.type(my_ttype)
quux
torch.split(quux, 1, dim=0)
tuple(map(torch.prod, torch.split(quux, 1, dim=0) ))

tensor([[1, 1, 1],
        [1, 0, 1]], dtype=torch.int8)

(tensor([[1, 1, 1]], dtype=torch.int8), tensor([[1, 0, 1]], dtype=torch.int8))

(tensor(1), tensor(0))

In [131]:
def agree_mat_t(A,B):
    '''
    Given two matrices (torch tensors) A::(n,m) and B::(n,m), 
    return C::(n,1) where 
    C[i] = 1 iff A[i] and B[i] agree at all indices
    and 0 otherwise.
    '''
    # (x == 0 or y == 0) or ((x != 0 and y != 0) and (x == y))
    A_unspecified = A == 0
    B_unspecified = B == 0
    A_or_B_unspecified = A_unspecified | B_unspecified
    
    A_specified = A != 0
    B_specified = B != 0
    A_and_B_specified = A_specified & B_specified
    A_equal_B = torch.eq(A,B)
    A_B_both_specified_and_equal = A_and_B_specified & A_equal_B

    ag = A_or_B_unspecified | A_B_both_specified_and_equal
#     return ag
#     result = np.prod(ag, axis=-1, dtype=myint)
    result = torch.zeros([A.shape[0]],dtype=my_dtype)
    result = torch.prod(ag, dim=1,dtype=my_dtype, out=result)
#     result = ag.type(torch.cuda.ByteTensor).all()
    return result#.type(my_torch_type)

In [132]:
%%timeit

agree_mat(random_pair_stack_a, random_pair_stack_b)

1.86 ms ± 5.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [133]:
random_pair_stack_a_t, random_pair_stack_b_t = t(random_pair_stack_a), t(random_pair_stack_b)

In [134]:
%%timeit

agree_mat_t(random_pair_stack_a_t, random_pair_stack_b_t)

687 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [135]:
np.array_equal(
    matrix_agree_result,
    agree_mat_t(t(random_pair_stack_a), t(random_pair_stack_b)).cpu().type(torch.int16).numpy()
)

True

### Union

In [136]:
def union_t(u, v):
#     if CAREFUL:
#         assert agree_(u,v)
    return torch.sign(u + v)

In [137]:
%%timeit

union(agreeing_pair_stack_a, agreeing_pair_stack_b)

63.7 µs ± 26.9 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [138]:
agreeing_pair_stack_a_t, agreeing_pair_stack_b_t = t(agreeing_pair_stack_a), t(agreeing_pair_stack_b)

In [139]:
%%timeit

union_t(agreeing_pair_stack_a_t, agreeing_pair_stack_b_t)

45.8 µs ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [140]:
np.array_equal(
    union(agreeing_pair_stack_a, agreeing_pair_stack_b),
    union_t(t(agreeing_pair_stack_a), t(agreeing_pair_stack_b)).cpu().type(torch.int16).numpy()
)

True

### Intersection

In [141]:
p_t = t(p); p_t
q_t = t(q); q_t

tensor([-1, -1,  0,  1,  1], dtype=torch.int8)

tensor([1, 0, 1, 1, 0], dtype=torch.int8)

In [142]:
def intersection_t(u, v):
    #u, v should be CharTensors (int8)
    
    s = u + v
    
    #torch.eq will always return a ByteTensor (uint8)
    e = torch.eq(u,v).type(torch.cuda.CharTensor)

    p = e * s
    
    result = torch.sign(p)
    return result
#     return torch.sign(  torch.eq(u, v) * (u + v) )

In [143]:
%%timeit

intersection(random_pair_stack_a, random_pair_stack_b)

333 µs ± 286 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [144]:
%%timeit

intersection_t(random_pair_stack_a_t, random_pair_stack_b_t)

115 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [145]:
np.array_equal(
    intersection(agreeing_pair_stack_a, agreeing_pair_stack_b),
    intersection_t(t(agreeing_pair_stack_a), t(agreeing_pair_stack_b)).cpu().type(torch.int16).numpy()
)

True

### Extension

In [148]:
def extension_t(pfv, O_t):
    return agree_mat_t(pfv, O_t)

In [149]:
%%timeit

list(map(lambda v: extension_(v, O), random_vectors))

1.18 s ± 7.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [150]:
%%timeit

np.apply_along_axis(lambda v: extension_(v, O), axis=1, arr=random_vectors).shape

1.45 s ± 9.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [151]:
random_vectors_t = t(np.array(random_vectors))

In [153]:
%%timeit

list(map(lambda v: extension_t(v, O_t), random_vectors_t))

10.9 s ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
%%timeit

torch.stack([extension_t(v, O_t) for v in torch.unbind(random_vectors_t, dim=0)])

10.6 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `pytorch` cell time is probably terribly slow because of the `list` and `map` (or list comprehension) operations happening in Python and on/involving the CPU...

In [155]:
%%timeit

extension_(choice(random_vectors), O)

12.8 µs ± 80.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [156]:
%%timeit

extension_t(choice(random_vectors_t), O_t)

114 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [157]:
%%timeit

for v in random_vectors:
    extension_(v, O)

1.13 s ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [158]:
%%timeit

for v in random_vectors_t:
    extension_t(v, O_t)

11 s ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [159]:
random_v = choice(random_vectors)

In [160]:
%%timeit

extension_(random_v, O)

11.3 µs ± 83.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [161]:
random_v_t = t(random_v)

In [162]:
%%timeit

extension_t(random_v_t, O_t)

105 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [163]:
for p in tqdm(random_vectors):
    assert np.array_equal(extension_(p, O), extension_t(t(p), O_t).cpu().type(torch.int16).numpy()), 'Disagreement on {0}: {1} vs. {2}'.format(p, extension_(p), extension_t(t(p)))

100%|██████████| 100000/100000 [00:22<00:00, 4474.75it/s]


### Conclusion

At a relatively low number of features (`m = 3`)...
 - `agreement` checking of entire matrices is ≈10x faster using `pytorch`+a GPU than using `numpy`.
 - `union` and `intersection` are about as fast in both implementations
 - `extension` seems abominably slower in pytorch than in numpy. Since `extension_t` is just a wrapper around `agree_mat_t`, and `agree_mat_t` is substantially faster than the numpy version, I think the explanation probably has something to do with inefficiencies in broadcasting. I'm not sure if there's anything I can do about that.